# dependencies

In [1]:
import sys
sys.path.insert(0, '../')  # Adjust the path as necessary
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from inverse_canopy import InverseCanopy

2024-02-11 09:10:49.331261: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# **Model Inputs**

In [4]:
from inverse_canopy import InverseCanopy
import tensorflow as tf
import numpy as np

tunable = {
 'num_samples': 100,      # number of monte carlo samples
 'learning_rate': 0.1,    # the gradient update rate
 'dtype': tf.float64,     # use 64-bit floats
 'epsilon': 1e-30,        # useful for avoiding log(0 + epsilon) type errors
 'max_steps': 5000,       # maximum steps, regardless of convergence
 'patience': 50,          # number of steps to wait before early stopping if the loss does not improve   
}

conditional_events = {
    'names': ['LCDL', 'PKRU', 'DHRS', 'DHRL'],
    'bounds': {
        'mean': {
            'min': 1e-14,
            'max': 1.00,
        },
        'std': {
            'min': 1e-10,
            'max': 1e8,
        },
     },
    'initial': {
       'mean': 5e-1,
       'std': 1e8,
    }
}

end_states = {
    'LCDL-1': {
        'sequence': [1, 0, 0, 0],
        'probability': 1.1e-8,
    },
    'LCDL-2': {
        'sequence': [1, 0, 0, 1],
        'probability': 1.00e-11,
    },
    'LCDL-3': {
        'sequence': [1, 0, 1, np.nan],
        'probability': 1.00e-11,
    },
    'LCDL-4': {
        'sequence': [1, 1, np.nan, np.nan],
        'probability': 1.00e-11,
    },
    'LCDL-0': {
        'sequence': [0, np.nan, np.nan, np.nan],
        'probability': 1.0 - 1.1e-8 - 1.00e-11 - 1.00e-11 - 1.00e-11, # one minus all the other end-states
    },
}

model = InverseCanopy(conditional_events, end_states, tunable)
model.fit(steps=tunable['max_steps'], patience=tunable['patience'], learning_rate=tunable['learning_rate'])
model.summarize(show_plot=True, show_metrics=False)

tunable initialized: dtype=<dtype: 'float64'>, epsilon=1e-30
learning_rate: 0.1,patience: 50,min_improvement: 0.001,max_steps: 5000,seed: 372


2024-02-11 09:10:52.308568: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: AMD Radeon Pro 5500M
2024-02-11 09:10:52.308594: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-02-11 09:10:52.308601: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 3.99 GB
2024-02-11 09:10:52.308642: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-11 09:10:52.308675: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-02-11 09:10:52.403279: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


InvalidArgumentError: Cannot assign a device for operation ReadVariableOp: Could not satisfy explicit device specification '' because the node {{colocation_node ReadVariableOp}} was colocated with a group of nodes that required incompatible device '/job:localhost/replica:0/task:0/device:GPU:0'. All available devices [/job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:GPU:0]. 
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=1 requested_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' assigned_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' resource_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignVariableOp: GPU CPU 
AssignSubVariableOp: CPU 
Identity: GPU CPU 
ReadVariableOp: GPU CPU 
_Arg: GPU CPU 

Colocation members, user-requested devices, and framework assigned devices, if any:
 

In [ ]:
tunable = {
 'num_samples': 1000,     # number of monte carlo samples
 'learning_rate': 0.1,    # the gradient update rate
 'dtype': tf.float64,     # use 64-bit floats
 'epsilon': 1e-30,        # useful for avoiding log(0 + epsilon) type errors
 'max_steps': 5000,       # maximum steps, regardless of convergence
 'patience': 50,          # number of steps to wait before stopping, if the loss does not improve
}

conditional_events = {
    'names': ['SDFR      ', 'LMFD      ', 'RFIR      ', 'LLRF      ', 'SSSD|~LLRF', 'SSSD|LLRF ', 'SYSO|~LLRF', 'SYSO|LLRF '],  # Assuming names based on the sequences
    'bounds': {
        'mean': {
            'min': 1e-14,
            'max': 1.00,
        },
        'std': {
            'min': 1e-10,
            'max': 1e8,
        },
     },
    'initial': {
       'mean': 5e-1,
       'std': 1e8,
    }
}

end_states = {
    'SDFR-0': {
        'sequence': [0, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
        'probability': 1 - 4.2e-3 - 4.2e-5 - 5.8e-9 - 2.1e-5 - 1.9e-5 - 2.1e-6 - 1.7e-8 - 4.3e-9,
    },
    'SDFR-1': {
        'sequence': [1, 0, 0, 0, 0, np.nan, np.nan, np.nan],
        'probability': 4.2e-3,
    },
    'SDFR-2': {
        'sequence': [1, 0, 0, 0, 1, np.nan, 0, np.nan]  ,
        'probability': 4.2e-5,
    },
    'SDFR-3': {
        'sequence': [1, 0, 0, 0, 1, np.nan, 1, np.nan],
        'probability': 5.8e-9,
    },
    'SDFR-4': {
        'sequence': [1, 0, 0, 1, np.nan, 0, np.nan, np.nan],
        'probability': 2.1e-5,
    },
    'SDFR-5': {
        'sequence': [1, 0, 0, 1, np.nan, 1, np.nan, 0],
        'probability': 1.9e-5,
    },
    'SDFR-6': {
        'sequence': [1, 0, 0, 1, np.nan, 1, np.nan, 1],
        'probability': 2.1e-6,
    },
    'SDFR-7': {
        'sequence': [1, 0, 1, np.nan, np.nan, np.nan, np.nan, np.nan],
        'probability': 1.7e-8,  
    },
    'SDFR-8': {
        'sequence': [1, 1, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
        'probability': 4.3e-9,  
    },
}

model = InverseCanopy(conditional_events, end_states, tunable)
model.fit(steps=tunable['max_steps'], patience=tunable['patience'], learning_rate=tunable['learning_rate'])
model.summarize(show_plot=True, show_metrics=False)